<a href="https://colab.research.google.com/github/mohammad2682/Covid-19/blob/master/CNN_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd
import cv2
import os
from matplotlib import pyplot as plt

In [2]:
!git clone https://github.com/mohammad2682/Covid19-Dataset

fatal: destination path 'Covid19-Dataset' already exists and is not an empty directory.


In [0]:
#importing train data from repo
import glob as glob
import skimage.transform as st
dl=['COVID','non-COVID']
dl=os.listdir('Covid19-Dataset/Covid-19-Dataset/train/')
xtrain=[]
p=0
for i in range(2):
  for fn in sorted(glob.glob('Covid19-Dataset/Covid-19-Dataset/train/'+dl[i]+'/*.png')):
    img = cv2.imread(fn)
    img=st.resize(img,(128,128))
    xtrain.append(img)

In [0]:
#importing test data from repo
dl=['COVID','non-COVID']
dl=os.listdir('Covid19-Dataset/Covid-19-Dataset/test/')
xtest=[]
p=0
for i in range(2):
  for fn in sorted(glob.glob('Covid19-Dataset/Covid-19-Dataset/test/'+dl[i]+'/*.png')):
    img = cv2.imread(fn)
    img=st.resize(img,(128,128))
    xtest.append(img)

In [0]:
#creating labels for train data
ytrain=[]
for i in range(2):
  for j in range(1000):
    ytrain.append(i)
#creating labels for test data    
ytest=[]
for j in range(252):
  ytest.append(0)
for j in range(230):
  ytest.append(1)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import BatchNormalization, Dropout

Using TensorFlow backend.


In [7]:
# Initialising the CNN
classifier = Sequential()
# Step 1 - Convolution
classifier.add(Conv2D(32, (7, 7), input_shape = (128, 128, 3), activation = 'relu'))
classifier.add(Dropout(0.3))
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Adding a second convolutional layer
classifier.add(Conv2D(64, (7, 7), activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
# Step 3 - Flattening
classifier.add(Flatten())
# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.3))
classifier.add(Dense(units = 1, activation = 'sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 122, 122, 32)      4736      
_________________________________________________________________
dropout_1 (Dropout)          (None, 122, 122, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 55, 55, 64)        100416    
_________________________________________________________________
dropout_2 (Dropout)          (None, 55, 55, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 27, 27, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 46656)            

# **Data Agumentation**

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
datagen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=False, 
    samplewise_center=False, 
    featurewise_std_normalization=False, 
    samplewise_std_normalization=False, 
    zca_whitening=False, 
    rotation_range=0, 
    width_shift_range=0.1, 
    height_shift_range=0.1, 
    horizontal_flip=True, 
    vertical_flip=False)

datagen.fit(xtrain)
datagen = ImageDataGenerator(rescale=1./255)
datagen.fit(xtest)

In [0]:
xtrain1=np.array(xtrain)
xtest1=np.array(xtest)
ytrain1=np.array(ytrain)
ytest1=np.array(ytest)

In [0]:
num_classes = 2
ytrain = keras.utils.to_categorical(ytrain, num_classes)
ytest  = keras.utils.to_categorical(ytest,  num_classes)

In [0]:
num_train = 2000
bs = 32
ep = 5
classifier.fit(xtrain1, ytrain1, epochs=ep, steps_per_epoch=num_train//bs, validation_data=(xtest1, ytest1), validation_steps = 482//bs)

Train on 2000 samples, validate on 482 samples
Epoch 1/5


In [35]:
ytrain.shape

(2000, 2)

In [36]:
xtrain.shape

AttributeError: ignored